# Simulating Shoreline Change using the Coastline Evolution Model (CEM)


<p style="text-align:center;"><img src="imgs/spit.jpg" alt="spit" width="500"/></p>

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import path
%matplotlib inline
import pandas as pd
from scipy import stats,signal
# from IPython.display import clear_output

from pymt.models import Cem, Waves
import buoypy as bp
import runmodels_functions as fun

## 1. Load Shoreline Data from CoastSAT & NDBC Buoy Wave Data

First, let's get our wave data:

### NDBC buoy

In [ ]:
# user input from NDBC buoy (make sure buoy has wave height available)
station = 45167
year = np.NAN
year_range = (2014,2023)

H = bp.historic_data(station,year,year_range)
X = H.get_all_stand_meteo()

# get rid of missing data
X.mask((X==99.0) | (X==9999.0), inplace=True)
#X.head(3) #preview first 3 lines of table

# plot relevant data to check it looks ok
%matplotlib inline
plotdata = True
if plotdata == True: fun.plotmeteo(X)

Extract relevant wave data for **CEM**

In [ ]:
Hs = np.mean(X.WVHT)
Tp = np.mean(X.DPD)
wdir = X.WDIR.values.copy()
wdir = wdir[~np.isnan(wdir)]
[Dir,count] = stats.mode(wdir) 

print('Mean significant wave height Hs (m)',Hs)
print('Mean wave period Tp (s)',Tp)
print('Mean wave direction (o)',Dir[0])

We will be using the significant wave height (Hs), wave period (Tp), and wave direction (Dir) data from the NDBC buoy as inputs into the WAVES model. 

### Shoreline

Now let's look at our shoreline:

In [ ]:
# gridded shoreline data
xy = np.loadtxt('data/Erie_example/pt_coords.txt')[:,:2]

x=xy[:,0]  # x-coords
y=xy[:,1]  # y-coords

## grid spacing in [meters]
dx=100 
dy=100

## if the number of data points is large (>10 thousand), reducing the size of the data by skipping every Z steps.
## here we are skipping every 200 data points to reduce the processing time when we regrid the shoreline
x,y = x[::200],y[::200] 

In [ ]:
def rotate_shoreline(x,y,angle='auto'):
    '''This function takes points (x,y) and rotates them by an angle
    The angle is automatically generated by the end points of the shorelines coordinates
    If this approach doesnt work for some reason you may input an angle maually (in degrees).'''
    
    if angle == 'auto':
        sign = -1 if y[-1]>y[0] else 1
        a = sign*np.arctan((y[-1]-y[0])/(x[-1]-x[0]))
    else:
        a = angle*np.pi/180
    # using this roation matix
    xnew = (x ) * np.cos(a) - (y ) * np.sin(a)  
    ynew = (x ) * np.sin(a) + (y ) * np.cos(a) 
    return xnew,ynew

xnew,ynew = rotate_shoreline(x,y)
x,y = xnew,ynew

def extend_shoreline(x,y,length):
    '''Extend the length of the shoreline by an amount, "length", on each end.'''
    XL = int(length)
    dx=length//100
    x = np.append(x,np.arange(x[-1],x[-1]+XL+dx,dx))
    y = np.append(y,np.ones([abs(len(y)-len(x))])*y[-1])
    x = np.append(np.flip(x),np.arange(x[0]-XL,x[0]-1,dx))
    y = np.append(np.flip(y),np.ones([abs(len(y)-len(x))])*y[0])
    return x,y

xnew,ynew = extend_shoreline(x,y,1e4)
x,y = xnew,ynew

In [ ]:
[xg,yg,elev] = fun.shorelinetogrid(x,y,dx,dy,plotdata=True);

Now we need to tidy up the rotated shoreline and also make the shore/shelf gradients smoother for CEM to calculate the shore/self slopes. The former is done by simply assigning all grid cells below the shorelines edge a "shoreline" grid value. The smoothing is done by a 2d convolution of the shoreface.

In [ ]:
def set_domain(pad=20):
    z_elev = elev.T # transpose of the elevations for CEM/plotting purposes

    z_elev[z_elev==-5] = -1 ## -5 is the "shoreline" flag from above.
    domain = -1*z_elev ## we want the domain to have values above sea-level to be >0 and below <0


    ##Trial and error steps:

    ## smooth the depths by averaging each cell in the domain by NxN of its neighbors
    neighs = np.ones([10,10])
    total = (10**2)
    domain = signal.convolve2d(domain,neighs/total,mode='same',boundary='symm')

    domain[z_elev==-1] = 1 ## our shoreline got smoothed out too. This fixes that and makes sure our land is at 1m above sea level


    N,M = domain.shape
    domnew = np.ones([N+pad,M]) ##here we add a "pad" to the bottom of the domain
    domnew[-N:,:] = domain # if this is undesirable, set "pad=0"
    domain = domnew
    
    return domain

domain= set_domain(pad=20)

In [ ]:
domain[20,:] = 1

In [ ]:
plt.imshow(domain[:21,:])

Let's visualize this updated coastline data with the function plot_coast. This will confirm that we have effectively translated the spit into model space

In [ ]:
fun.plot_coast(domain,dx,dy)

### 1.4 Shoreface/Shelf Slope

Our last step in data preparation is to find the average shoreface and shelf slope--two CEM parameters. As we were not able to determine this information from our CoastSat image, we need to derive these values ourselves. We will do this by appoximating a straight transect that extends from the beach off shore then using the equation from the last notebook to approximate the shelf slope:

In [ ]:
def find_shelf_slope(pad=20): #same pad as above
    h = lambda x: 0.1*x**(2/3) ## depth equation
    profile = np.copy(domain)[:,0] ## find a nice straight column in the domain...
    ## whose depth gradient is towards the top of the domain
    x = np.arange(len(profile))*dx ## the off shore coords in [meters]
    ## find the shoreline edge and approximate the gradient using the equation from the previous notebook:
    x0 = x[pad] ## edge of beach--defined by the pad from previous step if that was used
    xf = x[-1] ## open ocean depth
    return (h(xf)-h(x0))/(xf-x0) 


shelf_slope = find_shelf_slope() ##This should be on the order of 0.001. If not you may need to find this manually
# estimation of shoreface parameters:
shoreface_slope = 4*shelf_slope ## the shoreface is steeper than the shelf
shoreface_depth = 10 # depth meters

### 2. Model Setup
We are using the [Coastal Evolution Model (CEM)](https://csdms.colorado.edu/wiki/Model_help:CEM) and the [WAVES](https://csdms.colorado.edu/wiki/Model_help:Waves) models. The CEM simulates morphodynamic evolution of coastlines under varying wave climates. WAVES is the model that calculates the input wave energetics used in CEM as a function of wave period (T), deep water wave height (H<sub>0</sub>), and wave angle criteria. Run the help() commands on each function and read through their documentation.

In [ ]:
cem = Cem()
waves = Waves()

In [ ]:
## uncomment the lines below to see the model outlines

# help(cem)
# help(waves)

Check out the documentation online to get additional info on the parameters needed for each model:

[WAVES Documentation](https://csdms.colorado.edu/wiki/Model_help:Waves)

[CEM Documentation](https://csdms.colorado.edu/wiki/Model_help:CEM)

### 3.1. Initialize Models with Input Criteria

Here we are creating a _dictionary_ to assign value to our various input parameters for both the CEM and WAVES models. Defining CEM's parameter values this way facilitates transparency and easy manipulation of the variables. This dictionary will then be passed into the **initialize_models** function which sets up our models.

For our wave parameters, we are using the data from the "Extract Wave Data" section of the CoastSat notebook. Please see that file for more information on the data extraction.

*To convert wave direction to the **A** parameter, we use the following formula:*

$$\frac{|Dir-270|}{180}$$

*where **Dir** represents wave direction.*

In [ ]:
#convert wave direction to asymmetry term
Ap = 1-float(abs(Dir-270)/180)

## We are using 1-Ap because the waves in our domain are coming from the left.

In [ ]:
params = {
    
    ## CEM 
    'grid_spacing'    : dx,                #meters
    'shelf_slope'     : shelf_slope,
    'shoreface_depth' : shoreface_depth,   #meters
    'shoreface_slope' : shoreface_slope,
    'model__time_step': 1,               #days
    
    ##WAVES
    'wave_height' : Hs, #meters
    'wave_period' : Tp, #seconds
    'wave_angle_highness': 0.65, #important param
    'wave_angle_asymmetry': Ap
    
}

In the CEM, cells can either be water or land. Land cells will have a uniform height that is greater than the water depth. They will only be affected by the coastal processes of alongshore sediment transport and wave action. Inland cells will not be affected. Water cells will have a set bathymetry that will respond to sediment supply and wave action. The model calculates the bathymetry through the inner shelf slope (`shelf_slope`), depth at which the shoreface ends (``shoreface_depth``), and the shoreface slope (``shoreface_slope``). We can take a look at an example [here](Example_run___SPITS.ipynb) showing how the CEM runs without an input shoreline.

We initialize the model with the `initialize_models` function.

In [ ]:
fun.initialize_models(params,domain,cem,waves)

Let's look at our input coastline again with the **plot_coast** function as our baseline. 

We want to look at the input variable `land_surface__elevation` but this value is transformed into a 1d array in the model so we want to reshape it into the 2d array like our initial domain:

In [ ]:
land = cem.get_value('land_surface__elevation')
land = land.reshape(  domain.shape  )

fun.plot_coast(land,dx,dy)

## RUN THE MODEL!

Here comes the fun part. Run the model using our pre-written `run_model_loop function`.
The `run_model_loop function` will run the CEM and can display an animation of the evolving shoreline. The first input expresses the model run time (in years). The function displays an animation by default but you can set `animate=False` to decrease model computation time.


In [ ]:
T = 20 #years
fun.run_model_loop(T,domain,cem,waves,update_ani_years=1/2)

### Assignment #1 - The Shoreface

While not as dynamic as sandy shorelines, the shoreface is an important offshore transition region existing in between the surf zone and inner shelf that modulates sediment dynamics on the shoreline. In this first exercise, we will see how changing our shoreface geomorphology alters the evolution of our coastline through time.

To update specific parameters, we can overwrite their existing values in the `param` dictionary. Let's begin by altering our shoreface parameters:

In [ ]:
# insert your values below
params['shoreface_depth'] = 20 # recommended range: 10-20
params['shoreface_slope'] = 3*shelf_slope # recommended range: 3-10*inner shelf slope

In [ ]:
domain = set_domain()
domain[20,:] = 1
fun.initialize_models(params,domain,cem,waves)

In [ ]:
land = cem.get_value('land_surface__elevation')
land = land.reshape(  domain.shape  )

fun.plot_coast(land,dx,dy)

In [ ]:
T = 20 # model duration (years)

# with animation
#fun.run_model_loop(T,domain,cem,waves,update_ani_years=1/2)

# without animation
fun.run_model_loop(T,domain,cem,waves,animate=False)
land = cem.get_value('land_surface__elevation')
land = land.reshape(  domain.shape  )
fun.plot_coast(land,dx,dy)